# Create custom monitor in Watson OpenScale
**This is to showcase how to configure a custom monitor**

### how to use this notebook
- [ Build  `custmonitor`  library](#build_custmonitor)
- [Create a Package Extension in WML with custmonitor library](#package_extension)
- [Create a Software specification in WML with custmonitor ](#software_specifications)
- [Delete a Software specification in WML (optional) ](#delete_sw_package)
- [Load Custom monitor definition from model signature ](#load_custmonitor_definition)
- [Deploy Custom Metrics Provider ](#metrics_provider)
- [Test Custom Metrics Provider ](#test_metrics_provider)
- [Create Custom Monitor](#custom_monitor)
- [Feedback or Payload Logging](#feedback_loggigng)
- [Evaluate custom Monitor](#evaluate)
- [Delete custom Monitor](#delete)
---

### Set up Project Token and API Key  <a name="tokens"></a>
Before executing this notebook, you need to insert a project token and create a project data asset with config files :<br>


- `credentials_uploaded2cp4d.cfg` populated as described in [doc](/mlmonitor/README.md) section 2.1

#### Provide the project access token
1. When you imported the project from the github repo, the first cell of this notebook should contain the project access token.

2. If this notebook does not contain a first cell with a project access token, then to generate the token, click **More > Insert project token** on the menu bar.<br>

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)


#### chose model use case and deployment name here

In [ ]:
source_dir = 'use_case_gcr'
# Model for which the Custom monitor is attached (via subscription)
monitored_model = None

#### setup credentials for mlmonitor library

In [ ]:
import json
import os
import sys

cfg_filename='credentials_uploaded2cp4d.cfg'

if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D 
    mlmonitor_credentials = json.load(project.get_file(cfg_filename))


    with open("/tmp/credentials.cfg", "w") as f:
        json.dump(mlmonitor_credentials, f, indent=4, sort_keys=True)

    os.environ['MONITOR_CONFIG_FILE'] = "/tmp/credentials.cfg"

else:
     # RUN NOTEBOOK locally   

    with open(f"../{cfg_filename}", "r") as f:
        mlmonitor_credentials = json.load(f)

    os.environ['MONITOR_CONFIG_FILE'] = f"../{cfg_filename}"
    sys.path.append(os.path.abspath('../'))

In [ ]:
API_KEY = mlmonitor_credentials.get('saas').get('apikey')

#### install <i>mlmonitor</i>

In [ ]:
if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D
    !pip -q uninstall mlmonitor -y

In [ ]:
if 'PROJECT_ID' in os.environ:
    # RUN NOTEBOOK in CP4D
    !pip install mlmonitor[drift]

In [ ]:
from mlmonitor.src import logger,PROJECT_ROOT, IAM_URL, WOS_URL
from mlmonitor.src.wos import wos_client
from mlmonitor.src.utils.utils import send_alert
from mlmonitor.src.wml import wml_client,WML_SPACE_ID
from mlmonitor.src.wos.collect_alerts import collect_alerts
from mlmonitor.src import PROJECT_ROOT,IAM_URL
from mlmonitor.src.wml.package import create_package_extension,create_software_specification_extension
from mlmonitor.src.wos.cleanup_custom_monitor import cleanup_custom_monitor
from mlmonitor.src.wml.utils import get_function_uid_by_name,get_deployment_uid_by_name
from mlmonitor.src.wos.subscription import get_subscription_id_by_deployment
from mlmonitor.src.wos.data_mart import get_datamart_ids
from mlmonitor.src.wml.utils import get_deployment_uid_by_name

subscription_ids = get_subscription_id_by_deployment(wos_client=wos_client, deployment_name=monitored_model.strip())
data_marts = get_datamart_ids(wos_client=wos_client)

data_mart_id = data_marts[0]
subscription_id = subscription_ids[0]

wml_client.set.default_space(WML_SPACE_ID)

#### WML model

In [ ]:
from mlmonitor import WMLModelUseCase
model_uc = WMLModelUseCase(source_dir=source_dir, catalog_id=None, model_entry_id=None)

#### SM model

In [ ]:
from mlmonitor import SageMakerModelUseCase
model_uc = SageMakerModelUseCase(source_dir=source_dir, catalog_id=None, model_entry_id=None)

In [ ]:
model_uc.derive_model_states(monitored_model)

### Load  `custmonitor`  definition from model signature<a name="load_custmonitor_definition"></a>

In [ ]:
wml_function_provider = model_uc._model_config.custom_monitor_wml_function_provider

# Deployment name corresponds to the WML function deployed for this custom monitor
deployment_name = f"{wml_function_provider}-deploy"
py_fname = f"{wml_function_provider}-function"

# CUSTOM MONITOR SPECIFIC NAMES
provider_name = model_uc._model_config.custom_monitor_name

# Name Displayed in WOS UI
custom_monitor_name = model_uc._model_config.custom_monitor_provider_name

# custom_metrics_names = ("sensitivity", "specificity", "gender_less40_fav_prediction_ratio")
# custom_metrics_thresholds = (0.8, 0.6, 0.6)
custom_metrics_names = tuple(model_uc._model_config.custom_monitor_names)
custom_metrics_thresholds = tuple(model_uc._model_config.custom_monitor_thresholds)

print(f'Create a Custom Monitor for {source_dir} model use case:\n\n'
    f"Deployment Name {monitored_model}\n"
    f"Model use case  {source_dir}\n"
    f"Custom Metrics provider name [{provider_name}] to create\n"
    f"wml_function_provider {deployment_name}\n"
    f"Custom Monitor name [{custom_monitor_name}] to create\n"
    f"Custom metrics name [{custom_metrics_names}]\n"
    f"Custom metrics thresholds {custom_metrics_thresholds}"
)

### Build  `custmonitor`  library<a name="build_custmonitor"></a>

In [ ]:
PYTHON = sys.executable
!$PYTHON --version

In [ ]:
!cd $PROJECT_ROOT && $PYTHON $PROJECT_ROOT/setup.py sdist --formats=zip

In [ ]:
version=0.5
wml_client.software_specifications.get_uid_by_name(f"custmonitor-{version}")

In [ ]:
wml_client.software_specifications.list(limit=10)

In [ ]:
wml_client.package_extensions.list()

### Find existing Package Extension and SW Extension in WML with `custmonitor`  library<a name="package_extension_find"></a>

In [ ]:
from os.path import join
from os.path import join, exists, dirname

version=0.5
pkg_extn_name = f"custmonitor-{version}"
pkg_extn_description = "Pkg extension for Custom Monitor helpers"
pkg_extn_type = "pip_zip"
pkg_extn_path = join(PROJECT_ROOT, "dist", f"custmonitor-{version}.zip")
pkg_extn_uid = wml_client.package_extensions.get_uid_by_name(pkg_extn_name)
pkg_extn_uid

In [ ]:
sw_spec_name = f"custmonitor-{version}"
sw_sepc_decr = f"Software specification with custmonitor-{version}"
base_sw_spec = "runtime-22.1-py3.9"

sw_spec_uid = wml_client.software_specifications.get_uid_by_name(sw_spec_name)
sw_spec_uid

### Delete Software specification and Package Extension (optional)    <a name="delete_sw_package"></a>

In [ ]:
if sw_spec_uid != "Not Found":
    sw_spec_details = wml_client.software_specifications.delete(sw_spec_uid=sw_spec_uid)

if pkg_extn_uid != "Not Found":
    wml_client.package_extensions.delete(pkg_extn_uid)

### Create a Package Extension in WML with `custmonitor`  library<a name="package_extension"></a>

In [ ]:
if exists(pkg_extn_path):

    pkg_extn_uid, pkg_extn_url, details = create_package_extension(
        wml_client,
        pkg_extn_name,
        pkg_extn_description,
        pkg_extn_path,
        pkg_extn_type,
    )

    print(
        f"pkg_extn_uid : {pkg_extn_uid}, "
        f"pkg_extn_url : {pkg_extn_url}, "
        f"pkg_extn_details:\n{json.dumps(details, indent=4)}"
    )
else:
    details = wml_client.package_extensions.get_details(pkg_extn_uid)
    raise ValueError(f"{pkg_extn_path} not found with details:\n{details}")

### Create a Software specification in WML with `custmonitor` <a name="software_specifications"></a>

In [ ]:
sw_spec_uid = create_software_specification_extension(wml_client, pkg_extn_uid, sw_spec_name, sw_sepc_decr, base_sw_spec)
print(f"SW spec created with ID {sw_spec_uid}")

### Deploy Custom Metrics Provider (WML function) for Custom Monitor <a name="metrics_provider"></a>

In [ ]:
from mlmonitor.src.wml.deploy_custom_metrics_provider import deploy_custom_metrics_provider


def custom_metrics_provider(
    url=WOS_URL, apikey=API_KEY, use_case=model_uc.source_dir
):
    import importlib
    from custmonitor.metricsprovider.helpers import publish

    get_metrics = getattr(
        importlib.import_module(f"custmonitor.metrics.{use_case}"),
        "get_metrics",
    )

    def publish_to_monitor(input_data):
        response_payload = publish(
            input_data=input_data,
            url=url,
            apikey=apikey,
            get_metrics_fn=get_metrics,
        )
        return response_payload

    return publish_to_monitor

deploy_custom_metrics_provider(deployment_name=deployment_name,
                               function_code=custom_metrics_provider,
                               wml_space_id=WML_SPACE_ID,
                               python_function_name=py_fname,
                               runtime=f"custmonitor-{version}")

### Test Custom Metrics Provider <a name="test_metrics_provider"></a>

In [ ]:
input_data = {
    "input_data": [
        {
            "values": {
                "data_mart_id": data_mart_id,
                "subscription_id": subscription_id,
                "test": "test",
                "custom_monitor_run_id": "123",
                "custom_monitor_id": "not needed",
                "custom_monitor_instance_id": "not needed",
                "custom_monitor_instance_params": {
                    "custom_metrics_provider_id": "not needed",
                    "custom_metrics_wait_time": 300,
                },
            }
        }
    ]
}
    
deployment_uid = get_deployment_uid_by_name(wml_client=wml_client, deployment_name=deployment_name)
wml_client.deployments.score(deployment_uid, input_data)

### Create Custom Monitor in Watson OpenScale <a name="custom_monitor"></a>

In [ ]:
custom_monitor_config = {
'monitored_model':monitored_model.strip(),
 'wos_client':wos_client,
 'wml_client':wml_client,
 'deployment_name':deployment_name,
 'provider_name':provider_name,
 'custom_monitor_name':custom_monitor_name,
 'custom_metrics_names':custom_metrics_names,
 'custom_metrics_thresholds':custom_metrics_thresholds,
 'wml_space_id':WML_SPACE_ID,
 'apikey':API_KEY,
 'auth_url':IAM_URL
}
custom_monitor_config

In [ ]:
from mlmonitor.src.wos.configure_custom_monitor import configure_custom_monitor

custom_monitor_instance_details = configure_custom_monitor(**custom_monitor_config)

print(json.dumps(custom_monitor_instance_details.to_dict(), indent=4))

### Feedback or Payload Logging <a name="feedback_loggigng"></a>

In [ ]:
from mlmonitor.src.wos.run_feedback_logging import log_feedback_data

log_feedback_data(
    model_config=model_uc._model_config,
    deployment_name=monitored_model.strip(),
    deployment_target='aws',
    inference_samples=100,
    include_predictions=False,
)

### Evaluate custom Monitor <a name="evaluate"></a>

In [ ]:
from mlmonitor.src.wos.evaluate import evaluate_monitor
from mlmonitor.src import PROJECT_ROOT

evaluate_monitor(
    deployment_name=monitored_model.strip(),
    monitor_types=(custom_monitor_name.strip().lower(),),
)

### Delete Custom Monitor <a name="delete"></a>

In [ ]:
cleanup_custom_monitor(
    wos_client=wos_client,
    provider_name=provider_name,
    custom_monitor_name=custom_monitor_name,
    subscription_id=subscription_id,
    data_mart_id=data_mart_id,
)

### delete custom provider deployment

In [ ]:
deployment_uid = get_deployment_uid_by_name(wml_client=wml_client,deployment_name=deployment_name)
if deployment_uid:
    wml_client.deployments.delete(deployment_uid=deployment_uid)

### delete custom provider function

In [ ]:
function_uid = get_function_uid_by_name(wml_client=wml_client,function_name=py_fname)

if function_uid:
    wml_client.repository.delete(artifact_uid=function_uid)